In [21]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from IPython.display import Video
import plotly.express as px
import pandas as pd
import timeit
import os
import pickle
from tqdm import tqdm

In [49]:
WEBSITE = "../data/jre/website/"

In [22]:
from sklearn.cluster import MeanShift, estimate_bandwidth, KMeans, DBSCAN
from sklearn.neighbors.kde import KernelDensity
from scipy.signal import argrelextrema

def kde_cluster(Xdf, kernel="gaussian", bandwidth=10, plot=False):
    """
    Uses KDE to create clusters out of word vecs
    """
    X = np.array(Xdf['x'])
    X = X.reshape(-1, 1)

    # TODO There is probably bias at the boundaries, should mirror X
    kde = KernelDensity(kernel=kernel, bandwidth=bandwidth).fit(X)
    s = np.linspace(0, np.max(X)*1.5)
    e = kde.score_samples(s.reshape(-1, 1))

    # Reshape back to a 1 by N array
    X = X.reshape(1, -1)

    minima = argrelextrema(e, np.less)[0]
    # Use the linspace to convert back into word indexes
    minima = [s[m] for m in minima]
    # (0, minima 1), (minima 1, minima 2), ... (minima n-1, minima n), (minima n, end)
    minima_pairs = list(zip(np.insert(minima, 0, 0), np.append(minima, s[-1])))

    clusters = [
      np.unique(X[np.logical_and(X >= m1, X < m2)]) for m1, m2 in minima_pairs
    ]

    if plot:
      plt.plot(s, e)
      plt.show()
      print(f"Number of clusters: {len(clusters)}")
      for c in clusters:
          print("\t", len(c), np.unique([int(x) for x in c]))

    return clusters

# Store frame numbers in the clusters instead of the average of the last frame
def frame_idxs_clusters(avg_clusters, df):
    df['cluster'] = np.full_like((len(avg_clusters)), -1)

    for cluster_number, c in enumerate(avg_clusters):
        for frame_number, frame in enumerate(df['x']):
            if df['x'][frame_number] >= np.min(c) and df['x'][frame_number] <= np.max(c):
                df['cluster'][frame_number] = cluster_number
  
    return df

In [23]:
CACHE = "./jre-episodes.pickle"

with open(CACHE, "rb") as f:
    episodes = pickle.load(f)

print(f"Number of loaded episodes: {len(episodes)}")

Number of loaded episodes: 2455


In [24]:
# Main episodes vs main episodes with video averages
eps = [ep for ep in episodes if ep.video_averages is not None and ep.is_main_episode]
len([ep for ep in episodes if ep.is_main_episode]), len(eps)

(1463, 1373)

In [25]:
averages, total_frames = eps[0].video_averages

In [26]:
len(averages), total_frames

(12639, 239820)

In [27]:
from ipywidgets import interact, IntSlider
def show_frame_averages(x):
    df = pd.DataFrame({
      'x': eps[x].video_averages[0],
      'n': range(len(eps[x].video_averages[0])),
    })
    fig = px.scatter(df, x='x', color='n', title=str(eps[x]))
    fig.show()

interact(show_frame_averages, x=IntSlider(value=402, min=0, max=len(eps) - 1))

interactive(children=(IntSlider(value=402, description='x', max=1372), Output()), _dom_classes=('widget-intera…

<function __main__.show_frame_averages(x)>

In [28]:
# TODO How to estimate bandwidth for 2d data?
BANDWIDTH = 1

from ipywidgets import interact, IntSlider
def show_frame_averages(x):
    df = pd.DataFrame({
      'x': eps[x].video_averages[0],
      'n': range(len(eps[x].video_averages[0])),
    })
    avg_clusters = kde_cluster(df, plot=True, bandwidth=BANDWIDTH)

    df = frame_idxs_clusters(avg_clusters, df)
    # For a discrete color scale
    df['cluster'] = df['cluster'].astype(str)
    
    fig = px.scatter(df, x='x', color='cluster', title=str(eps[x]))
    fig.show()

interact(show_frame_averages, x=IntSlider(value=215, min=0, max=len(eps) - 1))

interactive(children=(IntSlider(value=215, description='x', max=1372), Output()), _dom_classes=('widget-intera…

<function __main__.show_frame_averages(x)>

### Generate Clusters For Each Episode

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

SKIP_DIVIS = 30 // 2 

episode_clusters = []
for ep in tqdm(eps):
    averages, total_frames = ep.video_averages
    df = pd.DataFrame({
      'x': averages,
      'n': range(len(averages)),
    })
    
    avg_clusters = kde_cluster(df, bandwidth=BANDWIDTH, plot=False)
    df = frame_idxs_clusters(avg_clusters, df)
    
    curr_cluster = 0
    clusters = {}
    prog_start = 0
    for i, cluster_num in enumerate(df['cluster']):
        progress = SKIP_DIVIS * i / total_frames
        if cluster_num != curr_cluster:
            arr = clusters.get(curr_cluster, [])
            arr.append((prog_start, progress))
            clusters[curr_cluster] = arr
            
            prog_start = progress
            curr_cluster = cluster_num
            
    episode_clusters.append((ep, clusters))    

In [38]:
CACHE = "./epside-clusters-cache.pkl"
with open(CACHE, "wb") as f:
    pickle.dump(episode_clusters, f)

In [50]:
# Generate pie data

pie_rows = []
for ep, cluster_timestamps in episode_clusters[:5]:
    data = {}
    for i, t in cluster_timestamps.items():
        data[i] = 0
        for start, end in t:
            data[i] += end - start

    pie_rows.append([ep.video_id, ep.title, ep.number, dict(data)])
            
pie_data = pd.DataFrame(pie_rows, columns=["id", "title", "number", "data"])
pie_data.to_csv(WEBSITE + "video_segments.csv")

In [52]:
#GET PICTURES FOR EACH CLUSTER

total = 0
for ep, cluster_timestamps in episode_clusters:
    # Get a frame for the middle of each cluster timestamp
    for i, t in cluster_timestamps.items():
        total += 1
total

10483